In [2]:
%matplotlib widget
import os
import sys
import math

# Путь к директории, где находится текущий ноутбук
NOTEBOOK_DIR = os.path.abspath(os.path.dirname("__file__"))  # работает в Jupyter

# Поднимаемся на один уровень вверх – получаем корень проекта
PROJECT_ROOT = os.path.abspath(os.path.join(NOTEBOOK_DIR, os.pardir, 'src'))

# Добавляем корень в начало sys.path, если его там ещё нет
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

In [3]:
from pathlib import Path

from afr3d.io.step_import import extract_solids, load_step

step_path = Path("../data/example_complex.stp")
shape_raw = load_step(step_path)
shape = extract_solids(shape_raw)

In [4]:
from afr3d.views.analytic_from_hlr import build_analytic_views_for_front_top_side, fuse_visibility_with_hlr
views = build_analytic_views_for_front_top_side(shape)

front_view = views["front"]

In [5]:
from afr3d.views.analytic_from_hlr import (
    _collect_hlr_polylines, _PointGridIndex
)
def fuse_visibility_with_hlr_DEBUG(
    shape,
    view,
    *,
    hlr_edge_samples: int = 16,
    tol: float | None = None,
    grid_size: int = 128,
):
    """
    Обновляет поля visibility / line_style у DraftEdge2D в AnalyticView2D
    по данным "чистого" HLR.

    Логика:
      1) строим HLR для этого вида (view.ax2);
      2) дискретизируем HLR-рёбра в 2D (visible_polys, hidden_polys);
      3) делаем spatial-index по всем HLR-точкам видимых и скрытых линий;
      4) для каждой точки аналитического ребра ищем ближайшую HLR-точку:
         - если dist < tol в visible: считаем "попадание в видимый HLR";
         - если dist < tol в hidden: считаем "попадание в скрытый HLR";
      5) по числу попаданий (vis_hits / hid_hits) классифицируем ребро:
         - vis>0, hid==0 -> "visible";
         - vis==0, hid>0 -> "hidden";
         - vis>0, hid>0  -> "silhouette";
         - оба 0         -> оставляем, как есть (по z-buffer).
      6) пересчитываем видимость вершин (visible=True, если есть хотя бы
         одно ребро visibility in {'visible', 'silhouette'}).

    Важно:
      - предполагается, что view.ax2 та же система, что и для выбора вида
        в HLR (мы именно её и используем).
    """
    # --- 0. Оценим масштаб вида, чтобы подобрать разумный tol ---
    all_pts = [(x, y) for e in view.edges.values() for (x, y) in e.points]
    if all_pts:
        xs = [p[0] for p in all_pts]
        ys = [p[1] for p in all_pts]
        xrange = max(xs) - min(xs)
        yrange = max(ys) - min(ys)
        scale = max(xrange, yrange, 1.0)
    else:
        scale = 1.0

    if tol is None:
        # допуск порядка 0.1% от габарита вида
        tol = 1e-3 * scale

    print(f"[FUSE] view scale ~ {scale:.3f}, tol = {tol:.6f}")
    
    # 1–2. HLR-полилинии
    visible_polys, hidden_polys = _collect_hlr_polylines(
        shape,
        view.ax2,
        n_samples=hlr_edge_samples,
    )

    visible_pts = [pt for poly in visible_polys for pt in poly]
    hidden_pts  = [pt for poly in hidden_polys  for pt in poly]

    print(f"[FUSE] HLR visible polylines: {len(visible_polys)}, points: {len(visible_pts)}")
    print(f"[FUSE] HLR hidden  polylines: {len(hidden_polys)}, points: {len(hidden_pts)}")

    vis_index = _PointGridIndex(visible_pts, grid_size=grid_size)
    hid_index = _PointGridIndex(hidden_pts,  grid_size=grid_size)

    # счётчики
    vis_only = 0
    hid_only = 0
    both = 0
    none = 0

    # 3. Обновляем видимость рёбер
    for e in view.edges.values():
        if not e.points:
            # счётчик
            none += 1
            continue

        vis_hits = 0
        hid_hits = 0

        for (x, y) in e.points:
            dv = vis_index.nearest_distance(x, y)
            dh = hid_index.nearest_distance(x, y)

            if dv < tol:
                vis_hits += 1
            if dh < tol:
                hid_hits += 1

        # нет попаданий ни в видимый, ни в скрытый HLR — оставляем как было
        if vis_hits == 0 and hid_hits == 0:
            # счётчик
            none += 1
            continue

        if vis_hits > 0 and hid_hits == 0:
            e.visibility = "visible"
            # счётчик
            vis_only += 1
        elif vis_hits == 0 and hid_hits > 0:
            e.visibility = "hidden"
            # счётчик
            hid_only += 1
        else:
            # есть попадания и туда, и туда — считаем, что это силует /
            # переходная зона; в практике HLR это редко, но при дискретизации
            # возможно
            e.visibility = "silhouette"
            # счётчик
            both += 1

        # line_style как и раньше
        if e.visibility == "visible":
            e.line_style = "solid"
        elif e.visibility == "hidden":
            e.line_style = "hidden"
        elif e.visibility == "silhouette":
            e.line_style = "solid_thick"
        else:
            e.line_style = "default"
    
    # счётчики
    print(f"[FUSE] edges: vis_only={vis_only}, hid_only={hid_only}, both={both}, none={none}")
    
    # 4. Пересчёт видимости вершин по обновлённым рёбрам
    for v in view.vertices.values():
        v.incident_edges.clear()
        v.visible = False

    for e_idx, e in view.edges.items():
        for v_idx in (e.v_start, e.v_end):
            if v_idx is None:
                continue
            if v_idx in view.vertices:
                view.vertices[v_idx].incident_edges.append(e_idx)

    visible_edge_states = {"visible", "silhouette"}
    for v in view.vertices.values():
        for e_idx in v.incident_edges:
            if view.edges[e_idx].visibility in visible_edge_states:
                v.visible = True
                break

In [6]:
from collections import Counter
# до слияния — статистика по z-buffer
print("=== BEFORE FUSE ===")
print("edges:", len(front_view.edges))
print("visibility:", Counter(e.visibility for e in front_view.edges.values()))

# сливаем с HLR
fuse_visibility_with_hlr_DEBUG(shape, front_view, hlr_edge_samples=16, tol=None)

print("=== AFTER FUSE ===")
print("edges:", len(front_view.edges))
print("visibility:", Counter(e.visibility for e in front_view.edges.values()))
print("line_style:", Counter(e.line_style for e in front_view.edges.values()))

=== BEFORE FUSE ===
edges: 393
visibility: Counter({'visible': 377, 'silhouette': 16})
[FUSE] view scale ~ 362.749, tol = 0.362749
[FUSE] HLR visible polylines: 321, points: 3288
[FUSE] HLR hidden  polylines: 244, points: 2602
[FUSE] edges: vis_only=0, hid_only=0, both=0, none=393
=== AFTER FUSE ===
edges: 393
visibility: Counter({'visible': 377, 'silhouette': 16})
line_style: Counter({'solid': 377, 'solid_thick': 16})


In [7]:
from afr3d.views.analytic_from_hlr import _collect_hlr_polylines, _PointGridIndex
from collections import Counter

front_view = views["front"]  # твой аналитический фронт

# 1. Масштаб вида (как в fuse)
all_pts = [(x, y) for e in front_view.edges.values() for (x, y) in e.points]
xs = [p[0] for p in all_pts]
ys = [p[1] for p in all_pts]
xrange_ = max(xs) - min(xs)
yrange_ = max(ys) - min(ys)
scale = max(xrange_, yrange_, 1.0)
tol = 1e-3 * scale
print(f"scale ≈ {scale:.3f}, tol ≈ {tol:.6f}")

# 2. Собираем HLR-полилинии
visible_polys, hidden_polys = _collect_hlr_polylines(
    shape,
    front_view.ax2,
    n_samples=16,
)
visible_pts = [pt for poly in visible_polys for pt in poly]
hidden_pts  = [pt for poly in hidden_polys  for pt in poly]
print(f"HLR visible pts: {len(visible_pts)}, hidden pts: {len(hidden_pts)}")

vis_index = _PointGridIndex(visible_pts, grid_size=128)
hid_index = _PointGridIndex(hidden_pts,  grid_size=128)

scale ≈ 362.749, tol ≈ 0.362749
HLR visible pts: 3288, hidden pts: 2602


In [8]:
# Возьмём несколько "подозрительных" рёбер (например, первые 5)
edges_list = list(front_view.edges.values())[:5]

for e in edges_list:
    print(f"\nEdge #{e.edge_index}, curve_type={e.curve_type}, faces={e.face_indices}")
    # посмотрим первые 10 точек
    for (x, y) in e.points[:10]:
        dv = vis_index.nearest_distance(x, y)
        dh = hid_index.nearest_distance(x, y)
        print(f"  point ({x:.3f},{y:.3f}) -> dv={dv:.4f}, dh={dh:.4f}")
    # для краткости только по первой точке:
    (x0, y0) = e.points[0]
    dv0 = vis_index.nearest_distance(x0, y0)
    dh0 = hid_index.nearest_distance(x0, y0)
    print(f"  first point dv0={dv0:.4f}, dh0={dh0:.4f}")


Edge #1, curve_type=line, faces=[1, 108]
  point (-6.894,-44.699) -> dv=inf, dh=inf
  point (-6.533,-44.812) -> dv=inf, dh=inf
  first point dv0=inf, dh0=inf

Edge #2, curve_type=line, faces=[1, 125]
  point (-6.533,-44.812) -> dv=inf, dh=inf
  point (-94.478,-42.184) -> dv=inf, dh=inf
  first point dv0=inf, dh0=inf

Edge #3, curve_type=line, faces=[1, 111]
  point (-94.478,-42.184) -> dv=inf, dh=inf
  point (-94.875,-42.061) -> dv=inf, dh=inf
  first point dv0=inf, dh0=inf

Edge #4, curve_type=circle, faces=[1, 96]
  point (-94.875,-42.061) -> dv=inf, dh=inf
  point (-94.516,-42.072) -> dv=inf, dh=inf
  point (-94.156,-42.084) -> dv=inf, dh=inf
  point (-93.795,-42.095) -> dv=inf, dh=inf
  point (-93.432,-42.106) -> dv=inf, dh=inf
  point (-93.069,-42.118) -> dv=inf, dh=inf
  point (-92.704,-42.129) -> dv=inf, dh=inf
  point (-92.339,-42.141) -> dv=inf, dh=inf
  point (-91.972,-42.152) -> dv=inf, dh=inf
  point (-91.604,-42.164) -> dv=inf, dh=inf
  first point dv0=inf, dh0=inf

Edge 

In [9]:
import math

from dataclasses import dataclass, field
from typing import Dict, List, Tuple, Optional, Set, Literal, Sequence
# Уплощаем HLR-полилинии в плоские облака точек
def _flatten_polylines(polylines: Sequence[Sequence[Tuple[float, float]]]
                       ) -> List[Tuple[float, float]]:
    """
    Превращаем список полилиний [[(x,y),...], ...] в плоский список точек.
    """
    pts: List[Tuple[float, float]] = []
    for poly in polylines:
        for (x, y) in poly:
            pts.append((float(x), float(y)))
    return pts


def _nearest_distance(x: float, y: float,
                      pts: Sequence[Tuple[float, float]]) -> float:
    """
    Наивный поиск ближайшей точки в облаке pts.
    Если pts пустой – возвращаем inf.
    """
    if not pts:
        return float("inf")
    best_d2 = float("inf")
    for px, py in pts:
        dx = px - x
        dy = py - y
        d2 = dx * dx + dy * dy
        if d2 < best_d2:
            best_d2 = d2
    return math.sqrt(best_d2)

In [10]:
def fuse_visibility_with_hlr_NEWREALISATION(
    view,
    hlr_visible_polylines: Sequence[Sequence[Tuple[float, float]]],
    hlr_hidden_polylines:  Sequence[Sequence[Tuple[float, float]]],
    *,
    samples_per_edge: int = 2,
    tol_scale: float = 1e-3,
) -> None:
    """
    Перезаписывает visibility / line_style рёбер в AnalyticView2D
    на основе HLR-результатов.
    Использует только 2D-координаты (ξ, η), т.е. предполагается,
    что HLR-полилинии уже выровнены в ту же систему coords, что и view.
    """

    # 1. Собираем облако HLR-точек
    vis_pts = _flatten_polylines(hlr_visible_polylines)
    hid_pts = _flatten_polylines(hlr_hidden_polylines)

    print(f"[FUSE] HLR visible pts: {len(vis_pts)}, hidden pts: {len(hid_pts)}")

    # 2. Оцениваем масштаб вида, чтобы задать разумный tol
    if view.vertices:
        xs = [v.x for v in view.vertices.values()]
        ys = [v.y for v in view.vertices.values()]
        dx = max(xs) - min(xs)
        dy = max(ys) - min(ys)
        scale = math.hypot(dx, dy)
    else:
        scale = 1.0

    tol = tol_scale * scale
    if tol <= 0:
        tol = 1e-3

    print(f"[FUSE] view scale ≈ {scale:.3f}, tol ≈ {tol:.6f}")

    # 3. Для статистики
    vis_only = hid_only = both = none = 0

    # 4. Проходим по рёбрам
    for e_idx, e in view.edges.items():
        if not e.points:
            continue

        # выбираем несколько репрезентативных точек на полилинии
        pts = e.points
        sample_indices = [0]
        if samples_per_edge >= 2 and len(pts) > 1:
            sample_indices.append(len(pts) // 2)

        dv = float("inf")
        dh = float("inf")

        for si in sample_indices:
            x, y = pts[si]
            dv = min(dv, _nearest_distance(x, y, vis_pts))
            dh = min(dh, _nearest_distance(x, y, hid_pts))

        # Классификация
        is_vis = dv < tol
        is_hid = dh < tol

        if is_vis and not is_hid:
            vis_only += 1
            # можно усилить видимость, но не обязательно
        elif is_hid and not is_vis:
            hid_only += 1
            # Переопределяем ребро как скрытое
            e.visibility = "hidden"
            e.line_style = "dashed"
        elif is_vis and is_hid:
            both += 1
            # неоднозначность – пока ничего не делаем
        else:
            none += 1
            # HLR молчит – оставляем как есть

    print(f"[FUSE] edges: vis_only={vis_only}, hid_only={hid_only}, both={both}, none={none}")

def debug_compare_view_and_hlr(view, hlr_visible_polylines, hlr_hidden_polylines):
    xs_view = [v.x for v in view.vertices.values()]
    ys_view = [v.y for v in view.vertices.values()]
    print(f"VIEW:  x=[{min(xs_view):.3f}, {max(xs_view):.3f}], "
          f"y=[{min(ys_view):.3f}, {max(ys_view):.3f}]")

    vis_pts = _flatten_polylines(hlr_visible_polylines)
    hid_pts = _flatten_polylines(hlr_hidden_polylines)
    all_x = [p[0] for p in vis_pts + hid_pts]
    all_y = [p[1] for p in vis_pts + hid_pts]
    print(f"HLR:   x=[{min(all_x):.3f}, {max(all_x):.3f}], "
          f"y=[{min(all_y):.3f}, {max(all_y):.3f}]")

In [14]:
from OCC.Core.gp import gp_Ax2
from OCC.Core.HLRBRep import HLRBRep_Algo, HLRBRep_HLRToShape
from OCC.Core.HLRAlgo import HLRAlgo_Projector
from OCC.Core.TopoDS import TopoDS_Shape
from OCC.Core.BRepAdaptor import BRepAdaptor_Curve
from OCC.Core.GeomAbs import GeomAbs_Line
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopAbs import TopAbs_EDGE

import matplotlib.pyplot as plt
def build_hlr_projection_ax2(shape: TopoDS_Shape, view_ax2: gp_Ax2):
    """
    Чистый HLR от OCCT для заданного gp_Ax2.
    Возвращает dict с shape'ами:
      visible, hidden, outline_visible, outline_hidden
    и тот же view_ax2.
    """
    algo = HLRBRep_Algo()
    algo.Add(shape)

    projector = HLRAlgo_Projector(view_ax2)
    algo.Projector(projector)
    algo.Update()
    algo.Hide()

    hlr_to_shape = HLRBRep_HLRToShape(algo)

    visible = hlr_to_shape.VCompound()
    hidden = hlr_to_shape.HCompound()

    try:
        outline_visible = hlr_to_shape.Rg1LineVCompound()
    except Exception:
        outline_visible = None

    try:
        outline_hidden = hlr_to_shape.Rg1LineHCompound()
    except Exception:
        outline_hidden = None

    return {
        "visible": visible,
        "hidden": hidden,
        "outline_visible": outline_visible,
        "outline_hidden": outline_hidden,
    }, view_ax2

In [16]:
from afr3d.views.analytic_from_hlr import _sample_hlr_compound_2d
# 1. Строим аналитический вид
views = build_analytic_views_for_front_top_side(shape)
front_view = views["front"]

# 2. Строим HLR для того же Ax2
hlr_proj, ax2 = build_hlr_projection_ax2(shape, front_view.ax2)

# 3. Собираем HLR-полилинии
vis_polys = []
hid_polys = []

for comp in [hlr_proj["visible"], hlr_proj["outline_visible"]]:
    vis_polys.extend(_sample_hlr_compound_2d(comp, ax2, n_samples=16))

for comp in [hlr_proj["hidden"], hlr_proj["outline_hidden"]]:
    hid_polys.extend(_sample_hlr_compound_2d(comp, ax2, n_samples=16))

# 4. Сливаем HLR с аналитикой
fuse_visibility_with_hlr_NEWREALISATION(
    front_view,
    hlr_visible_polylines = vis_polys,
    hlr_hidden_polylines  = hid_polys,
    samples_per_edge=2,
    tol_scale=1e-3
)

[FUSE] HLR visible pts: 3124, hidden pts: 1950
[FUSE] view scale ≈ 376.281, tol ≈ 0.376281
[FUSE] edges: vis_only=0, hid_only=0, both=0, none=393


In [21]:
debug_compare_view_and_hlr(
    front_view,
    hlr_visible_polylines = vis_polys,
    hlr_hidden_polylines  = hid_polys,
)

VIEW:  x=[-181.440, 181.440], y=[-49.756, 49.775]
HLR:   x=[12983.734, 13274.609], y=[4604.416, 4823.182]


# Адаптация hlr под кастомные функции проецирования

In [32]:
from typing import List, Tuple, Dict
from OCC.Core.gp import gp_Ax2
from OCC.Core.TopAbs import TopAbs_EDGE
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.BRepAdaptor import BRepAdaptor_Curve
from OCC.Core.HLRBRep import HLRBRep_Algo, HLRBRep_HLRToShape
from OCC.Core.HLRAlgo import HLRAlgo_Projector
from OCC.Core.TopoDS import TopoDS_Shape, topods

from afr3d.views.analytic import project_point_to_view_2d, DraftEdge2D

from OCC.Core.HLRBRep import HLRBRep_Algo, HLRBRep_HLRToShape
from OCC.Core.HLRAlgo import HLRAlgo_Projector
from OCC.Core.BRep import BRep_Tool
from OCC.Core.gp import gp_Ax2, gp_Pnt

# ... проект_point_to_view_2d у тебя уже есть ...


def _collect_2d_poly_from_hlr_edge(edge_3d, view_ax2: gp_Ax2, n_samples: int = 32):
    """
    Семплирует HLR-ребро по 3D-точкам и возвращает список (x, y) в координатах вида.
    edge_3d – TopoDS_Edge из HLRToShape (он уже лежит в плоскости проекции).
    """
    pts = []

    try:
        curve, first, last = BRep_Tool.Curve(edge_3d)
    except Exception:
        return pts

    if curve is None or last <= first:
        return pts

    if n_samples < 2:
        n_samples = 2

    du = (last - first) / (n_samples - 1)
    for i in range(n_samples):
        u = first + i * du
        try:
            p = curve.Value(u)  # gp_Pnt в 3D (на плоскости проекции)
        except Exception:
            continue

        x, y = project_point_to_view_2d(p, view_ax2)
        pts.append((x, y))

    return pts

def build_hlr_polylines_for_view(shape, view_ax2: gp_Ax2, n_samples: int = 32):
    """
    Запускает HLR для заданного вида и возвращает:
      hlr_visible_polylines: List[List[(x, y)]]
      hlr_hidden_polylines:  List[List[(x, y)]]
    где (x, y) – уже в координатах того же view_ax2, что и аналитический вид.
    """
    projector = HLRAlgo_Projector(view_ax2)  # ВАЖНО: тот же Ax2, что для аналитики
    algo = HLRBRep_Algo()
    algo.Add(shape)
    algo.Projector(projector)
    algo.Update()
    algo.Hide()

    hlr_shapes = HLRBRep_HLRToShape(algo)

    vis_compounds = [
        hlr_shapes.VCompound(),
        hlr_shapes.OutLineVCompound(),
        hlr_shapes.Rg1LineVCompound(),
        hlr_shapes.RgNLineVCompound(),
    ]
    hid_compounds = [
        hlr_shapes.HCompound(),
        hlr_shapes.OutLineHCompound(),
        hlr_shapes.Rg1LineHCompound(),
        hlr_shapes.RgNLineHCompound(),
    ]

    def collect_from_compound(cmpnd):
        polylines = []
        if cmpnd is None or cmpnd.IsNull():
            return polylines

        exp = TopExp_Explorer(cmpnd, TopAbs_EDGE)
        while exp.More():
            e = topods.Edge(exp.Current())
            pts = _collect_2d_poly_from_hlr_edge(e, view_ax2, n_samples=n_samples)
            if len(pts) >= 2:
                polylines.append(pts)
            exp.Next()
        return polylines

    hlr_visible_polylines = []
    hlr_hidden_polylines = []

    for c in vis_compounds:
        hlr_visible_polylines.extend(collect_from_compound(c))
    for c in hid_compounds:
        hlr_hidden_polylines.extend(collect_from_compound(c))

    return hlr_visible_polylines, hlr_hidden_polylines

def debug_compare_view_and_hlr(view_edges: Dict[int, DraftEdge2D],
                               hlr_visible_polylines,
                               hlr_hidden_polylines):
    # 1) AABB аналитического вида
    xs_view = []
    ys_view = []
    for e in view_edges.values():
        for (x, y) in e.points:
            xs_view.append(x)
            ys_view.append(y)

    # 2) AABB HLR (уже в координатах вида)
    xs_hlr = []
    ys_hlr = []
    for poly in hlr_visible_polylines + hlr_hidden_polylines:
        for (x, y) in poly:
            xs_hlr.append(x)
            ys_hlr.append(y)

    def bounds(xs, ys):
        if not xs or not ys:
            return None
        return min(xs), max(xs), min(ys), max(ys)

    b_view = bounds(xs_view, ys_view)
    b_hlr = bounds(xs_hlr, ys_hlr)

    print("VIEW:", end=" ")
    if b_view is None:
        print("empty")
    else:
        xmn, xmx, ymn, ymx = b_view
        print(f"x=[{xmn:.3f}, {xmx:.3f}], y=[{ymn:.3f}, {ymx:.3f}]")

    print("HLR: ", end=" ")
    if b_hlr is None:
        print("empty")
    else:
        xmn, xmx, ymn, ymx = b_hlr
        print(f"x=[{xmn:.3f}, {xmx:.3f}], y=[{ymn:.3f}, {ymx:.3f}]")

In [27]:
# 1. Строим аналитический вид
views = build_analytic_views_for_front_top_side(shape)
front_view = views["front"]

In [ ]:
hlr_vis, hlr_hid = build_hlr_polylines_for_view(shape, front_view.ax2)

print(f"[FUSE] HLR visible pts: {sum(len(p) for p in hlr_vis)}, "
      f"hidden pts: {sum(len(p) for p in hlr_hid)}")

debug_compare_view_and_hlr(front_view.edges, hlr_vis, hlr_hid)

[FUSE] HLR visible pts: 0, hidden pts: 0
VIEW: x=[-181.440, 181.440], y=[-49.778, 49.780]
HLR:  empty


In [37]:
# Вывод: путь пока нерешаем. Разница в системах координат
# Не получается выполнить корректный fuse